In [2]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional, Conv1D
from tensorflow.keras.optimizers import RMSprop, Adam
import spacy
from spacy_langdetect import LanguageDetector
import numpy as np
import random
import sys
import io
import requests
import re
import pandas
from tensorflow.keras import layers 
from tensorflow.keras import callbacks 
from langdetect import detect_langs


In [16]:
survey.head()

,Unnamed: 0,Unnamed: 0.1,book_authors,book_desc,book_title,genres,img,main_label
0,1098,1228,Joe Haldeman,The Earth's leaders have drawn a line in the i...,The Forever War,Science Fiction|Fiction|War,1228.jpg,Science Fiction
1,1248,1399,Chuck Palahniuk,Carl Streator is a reporter investigating Sudd...,Lullaby,Fiction|Horror|Contemporary|Fantasy|Novels,1399.jpg,Horror
2,2142,2408,Kim Harrison,Despite dating one vampire and living with ano...,For a Few Demons More,Fantasy|Urban Fantasy|Fantasy|Fantasy|Paranorm...,2408.jpg,Fantasy
3,2889,3243,Elizabeth Wurtzel,"A harrowing story of breakdowns, suicide attem...",Prozac Nation,Nonfiction|Autobiography|Memoir|Psychology|Hea...,3243.jpg,Biography
4,3115,3493,Jan-Philipp Sendker|Kevin Wiliarty,A poignant and inspirational love story set in...,The Art of Hearing Heartbeats,Fiction|Historical|Historical Fiction|Romance|...,3493.jpg,Historical


In [57]:
t=set()
v=set()
te=set()
s=set()


train=pandas.read_csv('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/train.txt', sep=',',)
i=0
while i<len(train):
    t.add(train.iloc[i,5])
    i+=1
    
validation=pandas.read_csv('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/valid.txt', sep=',',)
i=0
while i<len(validation):
    v.add(validation.iloc[i,5])
    i+=1
    
test=pandas.read_csv('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/test.txt', sep=',',)
i=0
while i<len(test):
    te.add(test.iloc[i,5])
    i+=1
    
survey=pandas.read_csv('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/survey.txt', sep=',',)
survey=survey.drop(columns=['Unnamed: 0'])
survey=survey.rename(columns={"Unnamed: 0.1": "Unnamed: 0"})
i=0
while i<len(survey):
    s.add(survey.iloc[i,5])
    i+=1
    
print (len(train),len(validation), len(test) , len(survey))


frames=[train, validation, test, survey]
books=pandas.concat(frames)
books

test1=train.merge(validation, left_on='img', right_on='img', how='inner')
test2=train.merge(test, left_on='img', right_on='img', how='inner')
test3=validation.merge(test, left_on='img', right_on='img', how='inner')
print (len(test1), len(test2), len(test3))
survey

31653 7926 3746 65
0 0 0


,Unnamed: 0,book_authors,book_desc,book_title,genres,img,main_label
0,1228,Joe Haldeman,The Earth's leaders have drawn a line in the i...,The Forever War,Science Fiction|Fiction|War,1228.jpg,Science Fiction
1,1399,Chuck Palahniuk,Carl Streator is a reporter investigating Sudd...,Lullaby,Fiction|Horror|Contemporary|Fantasy|Novels,1399.jpg,Horror
2,2408,Kim Harrison,Despite dating one vampire and living with ano...,For a Few Demons More,Fantasy|Urban Fantasy|Fantasy|Fantasy|Paranorm...,2408.jpg,Fantasy
3,3243,Elizabeth Wurtzel,"A harrowing story of breakdowns, suicide attem...",Prozac Nation,Nonfiction|Autobiography|Memoir|Psychology|Hea...,3243.jpg,Biography
4,3493,Jan-Philipp Sendker|Kevin Wiliarty,A poignant and inspirational love story set in...,The Art of Hearing Heartbeats,Fiction|Historical|Historical Fiction|Romance|...,3493.jpg,Historical
...,...,...,...,...,...,...,...
60,50135,Nagaru Tanigawa|Noizi Ito|Chris Pai,It's the end of the world as we know it - or i...,The Sigh of Haruhi Suzumiya,Novels|Light Novel|Fantasy|Science Fiction|Fic...,50135.jpg,Fantasy
61,50446,Dan Simmons,"Together in one volume for the first time, the...",The Hyperion Omnibus,Science Fiction|Fiction|Space|Space Opera|Fant...,50446.jpg,Science Fiction
62,51054,P.C. Cast,Demeter proposes that Lina exchange souls with...,Goddess of Spring,Romance|Fantasy|Fantasy|Mythology|Romance|Para...,51054.jpg,Romance
63,52108,Toni Blake,The last thing she needs right now is a manLin...,Letters to a Secret Lover,Romance|Contemporary Romance|Romance|Contempor...,52108.jpg,Romance


In [58]:
#books=pandas.read_csv('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/final_genres.txt', sep=',',)

#with open('C:/Users/benoi/Documents/HEC/_Hiver 2021/Données textuelles/last_words.txt', 'r') as file:
 #   last_words = file.read()

    
    
books=books.drop(['Unnamed: 0', 'book_authors'], axis=1)
books = books[books['book_desc'].notnull()]
books = books[books['book_title'].notnull()]
books



,book_desc,book_title,genres,img,main_label
0,Donate Body to Science. Check.When seventeen-y...,Freak of Nature,Young Adult|Science Fiction|Romance|Fantasy,14373.jpg,Young Adult
1,Percy und Annabeth haben es mit ihren Freunden...,Das Haus des Hades,Fantasy|Young Adult|Fantasy|Mythology,39253.jpg,Fantasy
2,"The grand residence of the famous alchemist, M...",Rose,Fantasy|Childrens|Middle Grade|Fantasy|Magic|C...,50647.jpg,Fantasy
3,✔Spell binding ✔Sexy ✔Romantic ✔Monster Roma...,My Beloved,Paranormal|Vampires|Fantasy|Supernatural|Roman...,4587.jpg,Fantasy
4,A.D. 1135. As church bells tolled for the deat...,When Christ and His Saints Slept,Historical|Historical Fiction|Fiction|Historic...,2775.jpg,Historical
...,...,...,...,...,...
60,It's the end of the world as we know it - or i...,The Sigh of Haruhi Suzumiya,Novels|Light Novel|Fantasy|Science Fiction|Fic...,50135.jpg,Fantasy
61,"Together in one volume for the first time, the...",The Hyperion Omnibus,Science Fiction|Fiction|Space|Space Opera|Fant...,50446.jpg,Science Fiction
62,Demeter proposes that Lina exchange souls with...,Goddess of Spring,Romance|Fantasy|Fantasy|Mythology|Romance|Para...,51054.jpg,Romance
63,The last thing she needs right now is a manLin...,Letters to a Secret Lover,Romance|Contemporary Romance|Romance|Contempor...,52108.jpg,Romance


In [59]:
i=0
cd=[]
ct=[]
while i<len(books):
    a=re.sub('[?,!@#*$:.«)(]', '', books.iloc[i,0].lower())
    cd.append(a)
    ct.append(re.sub('[?,!@#:*$.«)(]', '', books.iloc[i,1].lower()))
    
    i+=1
    
    
books['desc_clean']=cd
books['title_clean']=ct
books



,book_desc,book_title,genres,img,main_label,desc_clean,title_clean
0,Donate Body to Science. Check.When seventeen-y...,Freak of Nature,Young Adult|Science Fiction|Romance|Fantasy,14373.jpg,Young Adult,donate body to science checkwhen seventeen-yea...,freak of nature
1,Percy und Annabeth haben es mit ihren Freunden...,Das Haus des Hades,Fantasy|Young Adult|Fantasy|Mythology,39253.jpg,Fantasy,percy und annabeth haben es mit ihren freunden...,das haus des hades
2,"The grand residence of the famous alchemist, M...",Rose,Fantasy|Childrens|Middle Grade|Fantasy|Magic|C...,50647.jpg,Fantasy,the grand residence of the famous alchemist mr...,rose
3,✔Spell binding ✔Sexy ✔Romantic ✔Monster Roma...,My Beloved,Paranormal|Vampires|Fantasy|Supernatural|Roman...,4587.jpg,Fantasy,✔spell binding ✔sexy ✔romantic ✔monster roma...,my beloved
4,A.D. 1135. As church bells tolled for the deat...,When Christ and His Saints Slept,Historical|Historical Fiction|Fiction|Historic...,2775.jpg,Historical,ad 1135 as church bells tolled for the death o...,when christ and his saints slept
...,...,...,...,...,...,...,...
60,It's the end of the world as we know it - or i...,The Sigh of Haruhi Suzumiya,Novels|Light Novel|Fantasy|Science Fiction|Fic...,50135.jpg,Fantasy,it's the end of the world as we know it - or i...,the sigh of haruhi suzumiya
61,"Together in one volume for the first time, the...",The Hyperion Omnibus,Science Fiction|Fiction|Space|Space Opera|Fant...,50446.jpg,Science Fiction,together in one volume for the first time the ...,the hyperion omnibus
62,Demeter proposes that Lina exchange souls with...,Goddess of Spring,Romance|Fantasy|Fantasy|Mythology|Romance|Para...,51054.jpg,Romance,demeter proposes that lina exchange souls with...,goddess of spring
63,The last thing she needs right now is a manLin...,Letters to a Secret Lover,Romance|Contemporary Romance|Romance|Contempor...,52108.jpg,Romance,the last thing she needs right now is a manlin...,letters to a secret lover


In [60]:
from langdetect import detect_langs#remove list that are not in english
i=0

while i<len(books):   
    try:
        a=detect_langs(books.iloc[i,0])
        if str(a[0])[:2]=='en':
            i+=1
        else:
            books=books.drop(books.index[i])
       
    except :
         books=books.drop(books.index[i])
        

books

,book_desc,book_title,genres,img,main_label,desc_clean,title_clean
2,"The grand residence of the famous alchemist, M...",Rose,Fantasy|Childrens|Middle Grade|Fantasy|Magic|C...,50647.jpg,Fantasy,the grand residence of the famous alchemist mr...,rose
4,A.D. 1135. As church bells tolled for the deat...,When Christ and His Saints Slept,Historical|Historical Fiction|Fiction|Historic...,2775.jpg,Historical,ad 1135 as church bells tolled for the death o...,when christ and his saints slept
5,"Karl, aged seventeen, is hopelessly in love. B...",Dying to Know You,Young Adult|Contemporary|Realistic Fiction|Rom...,14235.jpg,Young Adult,karl aged seventeen is hopelessly in love but ...,dying to know you
7,"With this outrageous new novel, China Miéville...",Kraken,Fantasy|Fiction|Fantasy|Urban Fantasy|Science ...,29404.jpg,Fantasy,with this outrageous new novel china miéville ...,kraken
8,Both Tolkien and C.S. Lewis are literary super...,Tolkien and C.S. Lewis: The Gift of a Friendship,Biography|Nonfiction,50432.jpg,Biography,both tolkien and cs lewis are literary superst...,tolkien and cs lewis the gift of a friendship
...,...,...,...,...,...,...,...
59,"""Searing . . . captures an exquisite range of ...","Girl, Interrupted",Nonfiction|Autobiography|Memoir|Psychology,49758.jpg,Biography,"""searing captures an exquisite range of sel...",girl interrupted
60,It's the end of the world as we know it - or i...,The Sigh of Haruhi Suzumiya,Novels|Light Novel|Fantasy|Science Fiction|Fic...,50135.jpg,Fantasy,it's the end of the world as we know it - or i...,the sigh of haruhi suzumiya
61,"Together in one volume for the first time, the...",The Hyperion Omnibus,Science Fiction|Fiction|Space|Space Opera|Fant...,50446.jpg,Science Fiction,together in one volume for the first time the ...,the hyperion omnibus
62,Demeter proposes that Lina exchange souls with...,Goddess of Spring,Romance|Fantasy|Fantasy|Mythology|Romance|Para...,51054.jpg,Romance,demeter proposes that lina exchange souls with...,goddess of spring


In [61]:
i=0
b=[]
c=[]
g=[]
l=[]
while i<len(books):
    a=books.iloc[i,6].split(' ')
    l.append(a)
    g.append(books.iloc[i,4])
    j=0
    while j<len(a):
        b.append(a[j])
        j+=1
    c.append(len(a))
    i+=1
w=set(b)
dg=set(g)
books['title_list']=l
vocab=len(w)
print(vocab, max(c), len(g), len(b))
books

20249 41 35937 143631


,book_desc,book_title,genres,img,main_label,desc_clean,title_clean,title_list
2,"The grand residence of the famous alchemist, M...",Rose,Fantasy|Childrens|Middle Grade|Fantasy|Magic|C...,50647.jpg,Fantasy,the grand residence of the famous alchemist mr...,rose,[rose]
4,A.D. 1135. As church bells tolled for the deat...,When Christ and His Saints Slept,Historical|Historical Fiction|Fiction|Historic...,2775.jpg,Historical,ad 1135 as church bells tolled for the death o...,when christ and his saints slept,"[when, christ, and, his, saints, slept]"
5,"Karl, aged seventeen, is hopelessly in love. B...",Dying to Know You,Young Adult|Contemporary|Realistic Fiction|Rom...,14235.jpg,Young Adult,karl aged seventeen is hopelessly in love but ...,dying to know you,"[dying, to, know, you]"
7,"With this outrageous new novel, China Miéville...",Kraken,Fantasy|Fiction|Fantasy|Urban Fantasy|Science ...,29404.jpg,Fantasy,with this outrageous new novel china miéville ...,kraken,[kraken]
8,Both Tolkien and C.S. Lewis are literary super...,Tolkien and C.S. Lewis: The Gift of a Friendship,Biography|Nonfiction,50432.jpg,Biography,both tolkien and cs lewis are literary superst...,tolkien and cs lewis the gift of a friendship,"[tolkien, and, cs, lewis, the, gift, of, a, fr..."
...,...,...,...,...,...,...,...,...
59,"""Searing . . . captures an exquisite range of ...","Girl, Interrupted",Nonfiction|Autobiography|Memoir|Psychology,49758.jpg,Biography,"""searing captures an exquisite range of sel...",girl interrupted,"[girl, interrupted]"
60,It's the end of the world as we know it - or i...,The Sigh of Haruhi Suzumiya,Novels|Light Novel|Fantasy|Science Fiction|Fic...,50135.jpg,Fantasy,it's the end of the world as we know it - or i...,the sigh of haruhi suzumiya,"[the, sigh, of, haruhi, suzumiya]"
61,"Together in one volume for the first time, the...",The Hyperion Omnibus,Science Fiction|Fiction|Space|Space Opera|Fant...,50446.jpg,Science Fiction,together in one volume for the first time the ...,the hyperion omnibus,"[the, hyperion, omnibus]"
62,Demeter proposes that Lina exchange souls with...,Goddess of Spring,Romance|Fantasy|Fantasy|Mythology|Romance|Para...,51054.jpg,Romance,demeter proposes that lina exchange souls with...,goddess of spring,"[goddess, of, spring]"


In [62]:
i=0
cla=[]
itrain=[]
ival=[]
itest=[]
isurvey=[]
iautre=[]
while i<len(books):
    if books.iloc[i,3] in v:
        cla.append('v')
        ival.append(i)
    elif books.iloc[i,3] in t:
        cla.append('t')
        itrain.append(i)
    elif books.iloc[i,3] in te:
        cla.append('te')
        itest.append(i)
    elif books.iloc[i,3] in s:
        cla.append('s')
        isurvey.append(i)
    else:
        cla.append('a')
        iautre.append(i)
    i+=1

books['class']=cla


autre = books[books['class']=='a']

print (len(itrain), len(ival), len(itest), len(isurvey), len(iautre))
books['class'].value_counts()

#print (len(t), len(v))
#31661 7925 3749 65


27003 6133 2752 49 0


t     27003
v      6133
te     2752
s        49
Name: class, dtype: int64

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import collections
from numpy import array
from tensorflow.keras.utils import to_categorical


maxwords=max(c)
nbwords = collections.Counter(c)
print("Frequency of the elements in the List : ",nbwords)

#transformer x
x = books['title_list']
tra = books[books['class']=='t']['title_list']
val = books[books['class']=='v']['title_list']
tes = books[books['class']=='te']['title_list']
sur = books[books['class']=='s']['title_list']

xtoken = Tokenizer()
xtoken.fit_on_texts(x)

x_train = xtoken.texts_to_sequences(tra)
x_train = pad_sequences(x_train, maxlen=maxwords, padding='pre')

x_val = xtoken.texts_to_sequences(val)
x_val = pad_sequences(x_val, maxlen=maxwords, padding='pre')

x_test = xtoken.texts_to_sequences(tes)
x_test = pad_sequences(x_test, maxlen=maxwords, padding='pre')

x_survey = xtoken.texts_to_sequences(sur)
x_survey = pad_sequences(x_survey, maxlen=maxwords, padding='pre')

#transformer y
y = array(books['main_label'])
le = LabelEncoder()

y=le.fit_transform(y)
y = to_categorical(y)

lab2idx = dict(zip(le.classes_, range(len(le.classes_))))
idx2lab = {}
for k, v in lab2idx.items():
    idx2lab[v] = k

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)

print (x_train.shape, x_test.shape, y_train.shape, y_test.shape, vocab, idx2lab)
print(x[0])

In [71]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import collections
from numpy import array
from tensorflow.keras.utils import to_categorical


maxwords=max(c)
nbwords = collections.Counter(c)
#print("Frequency of the elements in the List : ",nbwords)

#transformer x
x = books['title_list']
xtoken = Tokenizer()
xtoken.fit_on_texts(x)
x = xtoken.texts_to_sequences(x)
x = pad_sequences(x, maxlen=maxwords, padding='pre')

#transformer y
y = array(books['main_label'])
le = LabelEncoder()

y=le.fit_transform(y)
y = to_categorical(y)

lab2idx = dict(zip(le.classes_, range(len(le.classes_))))
idx2lab = {}
for k, v in lab2idx.items():
    idx2lab[v] = k

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)

i=0
x_train = []
while i<len(itrain):
    j=itrain[i]
    x_train.append(x[j])
    i+=1
x_train=np.asarray(x_train)  

i=0
x_val = []
while i<len(ival):
    j=ival[i]
    x_val.append(x[j])
    i+=1
x_val=np.asarray(x_val)  

i=0
x_test = []
while i<len(itest):
    j=itest[i]
    x_test.append(x[j])
    i+=1
x_test=np.asarray(x_test)

i=0
x_survey = []
while i<len(isurvey):
    j=isurvey[i]
    x_survey.append(x[j])
    i+=1
x_survey=np.asarray(x_survey)




i=0
y_train = []
while i<len(itrain):
    j=itrain[i]
    y_train.append(y[j])
    i+=1
y_train=np.asarray(y_train)  

i=0
y_val = []
while i<len(ival):
    j=ival[i]
    y_val.append(y[j])
    i+=1
y_val=np.asarray(y_val)  

i=0
y_test = []
while i<len(itest):
    j=itest[i]
    y_test.append(y[j])
    i+=1
y_test=np.asarray(y_test)

i=0
y_survey = []
while i<len(isurvey):
    j=isurvey[i]
    y_survey.append(y[j])
    i+=1
y_survey=np.asarray(y_survey)

print(x_train.shape, x_val.shape, x_test.shape, x_survey.shape)
print(y_train.shape, y_val.shape, y_test.shape, y_survey.shape)


(27003, 41) (6133, 41) (2752, 41) (49, 41)
(27003, 13) (6133, 13) (2752, 13) (49, 13)


In [10]:
from sklearn.utils import class_weight
import numpy as np
   

#cw = class_weight.compute_class_weight('balanced',np.unique(books['main_label']), books['main_label']),


y_integers = np.argmax(y, axis=1)
cw = class_weight.compute_class_weight('balanced', np.unique(y_integers), y_integers)
cw = dict(enumerate(cw))

print(cw, books['main_label'].value_counts())
print(len(y_integers))

{0: 1.6211790393013101, 1: 2.0040485829959516, 2: 0.3844162567952369, 3: 0.7638888888888888, 4: 2.5384615384615383, 5: 0.8240843507214206, 6: 3.877284595300261, 7: 1.5714285714285714, 8: 1.164249313994512, 9: 0.5062212374296915, 10: 1.3089466725429704, 11: 2.1444043321299637, 12: 0.7105263157894737} Fantasy                  7726
Romance                  5867
Young Adult              4180
Historical               3888
Mystery & Crime          3604
Religion & Philosophy    2551
Science Fiction          2269
Politics & History       1890
Biography                1832
Childrens                1482
Sequential Art           1385
Horror                   1170
Poetry                    766
Name: main_label, dtype: int64
38610


In [12]:
j=0
embedding_vector = {}
glove = open('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/glove.6B.300d.txt', encoding="utf8")
for line in glove:
    a = line.split(' ')
    word = a[0]
    coef = np.array(a[1:],dtype = 'float32')
    embedding_vector[word] = coef
    
embedding_matrix = np.zeros((vocab+1,300))
for word,i in xtoken.word_index.items():
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value
    else:
        j+=1
        
print(embedding_matrix.shape, j, 'Pourcentage de mots non trouvés dans GLOVE: ', j/embedding_matrix.shape[0])

(23139, 300) 7007 Pourcentage de mots non trouvés dans GLOVE:  0.3028220752841523


In [13]:
model = Sequential()
model.add(Embedding(vocab+1, 300, weights = [embedding_matrix], input_length=maxwords, trainable = True))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))#filter et kernel
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size=2))#poolsize
model.add(Bidirectional(LSTM(100)))#100
model.add(Dropout(0.25))#dropout
model.add(Dense(100,activation = 'relu'))
model.add(Dense(len(dg),activation = 'sigmoid'))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics = ['accuracy'])
ES= callbacks.EarlyStopping(monitor='loss', patience=4)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 41, 300)           6941700   
_________________________________________________________________
conv1d (Conv1D)              (None, 41, 32)            28832     
_________________________________________________________________
batch_normalization (BatchNo (None, 41, 32)            128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 20, 32)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               106400    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               2

In [43]:
history = model.fit(x_train,y_train, epochs = 20, validation_data=(x_val, y_val), callbacks=[ES], verbose=1, batch_size=128, class_weight=cw)

Epoch 1/20
203/203 [==============================] - 36s 161ms/step - loss: 0.1358 - accuracy: 0.9323 - val_loss: 4.1193 - val_accuracy: 0.4138
Epoch 2/20
203/203 [==============================] - 31s 153ms/step - loss: 0.1194 - accuracy: 0.9399 - val_loss: 4.1761 - val_accuracy: 0.3989
Epoch 3/20
203/203 [==============================] - 32s 160ms/step - loss: 0.1260 - accuracy: 0.9355 - val_loss: 4.2564 - val_accuracy: 0.3992
Epoch 4/20
203/203 [==============================] - 31s 155ms/step - loss: 0.1233 - accuracy: 0.9384 - val_loss: 4.2786 - val_accuracy: 0.4096
Epoch 5/20
203/203 [==============================] - 30s 150ms/step - loss: 0.1208 - accuracy: 0.9385 - val_loss: 4.3252 - val_accuracy: 0.3992


In [31]:
modelCNN = Sequential()
modelCNN.add(Embedding(vocab+1, 300, weights = [embedding_matrix], input_length=maxwords, trainable = True))#maxwords
modelCNN.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))#a#<ll
modelCNN.add(layers.BatchNormalization())
modelCNN.add(layers.MaxPooling1D(pool_size=2))#poolsize
modelCNN.add(layers.Flatten())
modelCNN.add(Dense(len(dg),activation = 'sigmoid'))#vs relu
optimizer=Adam(learning_rate=0.0001,epsilon=0.1)#lr
modelCNN.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics = ['accuracy'])
ES= callbacks.EarlyStopping(monitor='loss', patience=3)

print(modelCNN.summary())



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 41, 300)           6941700   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 41, 32)            28832     
_________________________________________________________________
batch_normalization_4 (Batch (None, 41, 32)            128       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 20, 32)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 13)                8333      
Total params: 6,978,993
Trainable params: 6,978,929
Non-trainable params: 64
___________________________________________

In [39]:
optimizer=Adam(learning_rate=0.001,epsilon=0.1)
modelCNN.compile(optimizer='Adam',loss='categorical_crossentropy', metrics = ['accuracy'])

In [41]:
ES5= callbacks.EarlyStopping(monitor='loss', patience=5)

historyCNN = modelCNN.fit(x_train,y_train, epochs = 25, validation_data=(x_val, y_val), callbacks=[ES5], verbose=1, batch_size=128, class_weight=cw)

Epoch 1/25
203/203 [==============================] - 25s 117ms/step - loss: 0.2899 - accuracy: 0.8637 - val_loss: 3.8554 - val_accuracy: 0.3869
Epoch 2/25
203/203 [==============================] - 24s 121ms/step - loss: 0.2628 - accuracy: 0.8761 - val_loss: 3.9726 - val_accuracy: 0.3819
Epoch 3/25
203/203 [==============================] - 25s 124ms/step - loss: 0.2412 - accuracy: 0.8856 - val_loss: 4.0350 - val_accuracy: 0.3812
Epoch 4/25
203/203 [==============================] - 25s 125ms/step - loss: 0.2256 - accuracy: 0.8942 - val_loss: 4.0724 - val_accuracy: 0.3867
Epoch 5/25
203/203 [==============================] - 26s 128ms/step - loss: 0.2224 - accuracy: 0.8987 - val_loss: 4.2448 - val_accuracy: 0.3599
Epoch 6/25
203/203 [==============================] - 27s 131ms/step - loss: 0.2168 - accuracy: 0.9001 - val_loss: 4.2482 - val_accuracy: 0.3800
Epoch 7/25
203/203 [==============================] - 27s 133ms/step - loss: 0.2138 - accuracy: 0.9042 - val_loss: 4.3638 - val_ac

[0 0 0 0 0 0 0 1 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


(38613, 41)

In [15]:
def prediction(sentence, modele=model):
    sentence = sentence.lower()
    sentence=sentence.split(' ')
    test = xtoken.texts_to_sequences(sentence)
    flat_list =  [[item for elem in test for item in elem]]
    bob = pad_sequences(flat_list, maxlen=41, padding='pre')
    a=modele.predict(bob)
    b=np.argmax(a, axis=1)
    pred2 = np.argsort(a,axis=1)[:,-2]
    pred3 = np.argsort(a,axis=1)[:,-3]
    genre=idx2lab[b[0]]
    return (genre, a, pred2 pred3)
    



In [24]:
prediction("love murder")

('Romance',
 array([[0.4749476 , 0.01452982, 0.53089595, 0.10134837, 0.8884505 ,
         0.8770132 , 0.04750389, 0.10779369, 0.16176668, 0.9239906 ,
         0.53383493, 0.44093302, 0.8590435 ]], dtype=float32))

In [209]:
# ne pas exécuter
sentence='I love you'
sentence = sentence.lower()
sentence=sentence.split(' ')
test = xtoken.texts_to_sequences(sentence)
flat_list =  [[item for elem in test for item in elem]]
bob = pad_sequences(flat_list, maxlen=41, padding='pre')
print (sentence, test, flat_list, bob)

['i', 'love', 'you'] [[16], [11], [15]] [[16, 11, 15]] [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0 16 11 15]]


In [176]:
from sklearn.metrics import classification_report
print(classification_report( y_train, predicted_classes, target_names=class_names))

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,   319,     1,   168, 10218,     8,    35,     4,
         197,     5,     3,  6958, 10219])

In [44]:
model.save('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/CNN-LSTM')

INFO:tensorflow:Assets written to: C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/CNN-LSTM\assets


INFO:tensorflow:Assets written to: C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/CNN-LSTM\assets
